In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
import copy
import gc
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook
import json
import random
random.seed(4)
%matplotlib inline

In [0]:
data_folder = './drive/My Drive/Boosters OneTwoTrip/share_data/'
subm_folder = './drive/My Drive/Boosters OneTwoTrip/Task 2/submissions'

In [0]:
# считываем исходные данные
df_train = pd.read_csv(os.path.join(data_folder, 'onetwotrip_challenge_train.csv'))
df_test = pd.read_csv(os.path.join(data_folder,'onetwotrip_challenge_test.csv'))

In [0]:
# переводим хэшированные userid в int
all_user_ids = list(df_train["userid"].unique()) + list(df_test["userid"].unique())
all_user_ids_dict = dict(zip(all_user_ids, range(len(all_user_ids))))

df_train['userid'] = df_train['userid'].map(all_user_ids_dict)
df_test['userid'] = df_test['userid'].map(all_user_ids_dict)

In [0]:
# отстортировываем данные по времени внутри каждого юзера (field4 - номер заказа юзера)
df_train.sort_values(['userid', 'field4'], inplace=True)
df_test.sort_values(['userid', 'field4'], inplace=True)

### Первая часть признаков (до НГ)

In [0]:
# небольшое преобразование field1 и field14
# посокольку эти признаки нормализованы, была предпринята попытка вернуть им исходный вид
df_train['field1_1'] = round(df_train['field1']*6445.62 + 8538,-2)
df_test['field1_1'] = round(df_test['field1']*6445.62 + 8538,-2)

df_train['field14_1'] = round(df_train['field14']*357 + 2686,-2)
df_test['field14_1'] = round(df_test['field14']*357 + 2686,-2)

In [0]:
# создание признака часов до вылета (field16 - количество дней до вылета с момента покупки)
def get_hours(field16, hour_buy, hour_flight):
    if field16==0:
        return hour_flight-hour_buy
    else:
        return (24-hour_buy) + hour_flight +(field16-1)*24

df_train['hours_to_flight'] = df_train.apply(lambda x: get_hours(x['field16'], x['field11'], x['field23']), axis=1) 
df_test['hours_to_flight'] = df_test.apply(lambda x: get_hours(x['field16'], x['field11'], x['field23']), axis=1)

In [0]:
# field0 - дней с предыдущей покупки. Заменим 0 на None для первичников в этом признаке
df_train.loc[df_train.field5==1, 'field0'] = np.nan
df_test.loc[df_test.field5==1, 'field0'] = np.nan

In [0]:
fields_cols = list(filter(lambda x: 'field' in x, df_train.columns))
ind_cols = list(filter(lambda x: 'indicator' in x, df_train.columns))
cols_for_shift = fields_cols + ind_cols

In [0]:
# смотрим какие признаки были в предыдущей и следующей покупки
name_cols_shift1 = ['shift1_'+col for col in cols_for_shift]
name_cols_shiftminus = ['shiftminus_'+col for col in cols_for_shift]

df_train[name_cols_shift1] = df_train.groupby('userid')[cols_for_shift].shift(1)
df_test[name_cols_shift1] = df_test.groupby('userid')[cols_for_shift].shift(1)
for col, shift_col in zip(cols_for_shift, name_cols_shift1):
    df_train[col+'_dif'] = df_train[col] - df_train[shift_col]
    df_test[col+'_dif'] = df_test[col] - df_test[shift_col]

df_train[name_cols_shiftminus] = df_train.groupby('userid')[cols_for_shift].shift(-1)
df_test[name_cols_shiftminus] = df_test.groupby('userid')[cols_for_shift].shift(-1)
for col, shift_col in zip(cols_for_shift, name_cols_shiftminus):
    df_train[col+'_minusdif'] = df_train[col] - df_train[shift_col]
    df_test[col+'_minusdif'] = df_test[col] - df_test[shift_col]

In [0]:
# mean encoding всех goal по field12
df_train['field12_cut'] = pd.cut(df_train['field12'], [0.999,2,3,4,5,8,9,15,25,500])
df_test['field12_cut'] = pd.cut(df_test['field12'], [0.999,2,3,4,5,8,9,15,25,500])
for targ in ['goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1']:
    map_dict = dict(df_train.groupby('field12_cut')[targ].mean())

    df_train['field12_mean_encoding_'+targ] = df_train['field12_cut'].map(map_dict).astype(np.float64)
    df_test['field12_mean_encoding_'+targ] = df_test['field12_cut'].map(map_dict).astype(np.float64)

df_train.drop('field12_cut', axis=1, inplace=True)
df_test.drop('field12_cut', axis=1, inplace=True)

In [0]:
# признак - когда в последний раз field6 был не нулем
df_train['field6_mod'] = df_train['shift1_field6'].apply(lambda x: 1 if x >0 else x)
df_train['field6_strange_feature'] = (df_train['field6_mod']* df_train['field4']-1).fillna(0).replace({-1:None}).fillna(method='ffill')
df_train.drop(['field6_mod'], axis=1)
df_train['field6_strange_feature_dif'] = df_train['field6_mod'] - df_train['field6'] 

df_test['field6_mod'] = df_test['shift1_field6'].apply(lambda x: 1 if x >0 else x)
df_test['field6_strange_feature'] = (df_test['field6_mod']* df_test['field4']-1).fillna(0).replace({-1:None}).fillna(method='ffill')
df_test.drop(['field6_mod'], axis=1)
df_test['field6_strange_feature_dif'] = df_test['field6_mod'] - df_test['field6'] 

In [0]:
# ??? (не помню, что я сделал и зачем, но этот признак есть в итоговм решении)
df_train['field0_mod'] = df_train['shift1_field0'].apply(lambda x: 1 if x >0 else x)
df_train['field0_strange_feature'] = (df_train['field0_mod']* df_train['field4']-1).fillna(0).replace({-1:None}).fillna(method='ffill')
df_train.drop(['field0_mod'], axis=1)
df_train['field0_strange_feature_dif'] = df_train['field0_mod'] - df_train['field0'] 

df_test['field0_mod'] = df_test['shift1_field0'].apply(lambda x: 1 if x >0 else x)
df_test['field0_strange_feature'] = (df_test['field0_mod']* df_test['field4']-1).fillna(0).replace({-1:None}).fillna(method='ffill')
df_test.drop(['field0_mod'], axis=1)
df_test['field0_strange_feature_dif'] = df_test['field0_mod'] - df_test['field0'] 

In [0]:
# разница между месяцом покпуки и месяцом вылета
df_train['field2_3_dif'] = df_train[['field2', 'field3']].apply(lambda r: r['field3']-r['field2'] if r['field3']>=r['field2'] else 12-r['field2']+r['field3'], axis=1)
df_test['field2_3_dif'] = df_test[['field2', 'field3']].apply(lambda r: r['field3']-r['field2'] if r['field3']>=r['field2'] else 12-r['field2']+r['field3'], axis=1)

In [0]:
# признаки отношения и разницы field12 и field16
df_train['field_12_16_div'] = df_train['field12']/df_train['field16']
df_train['field_12_16_dif'] = df_train['field12'] - df_train['field16']
df_test['field_12_16_div'] = df_test['field12']/df_test['field16']
df_test['field_12_16_dif'] = df_test['field12'] - df_test['field16']

In [0]:
# признаки отношения field1 и field14
df_train['field_1_14_1_div'] = df_train['field1_1']/df_train['field14_1']
df_test['field_1_14_1_div'] = df_test['field1_1']/df_test['field14_1']

df_train['field_1_14_div'] = df_train['field1']/df_train['field14']
df_test['field_1_14_div'] = df_test['field1']/df_test['field14']

In [0]:
# среднее field12 и field16 по userid 
df_train[['field16_mean', 'field12_mean']] = df_train.groupby('userid')['field16', 'field12'].transform(np.mean)
df_test[['field16_mean', 'field12_mean']] = df_test.groupby('userid')['field16', 'field12'].transform(np.mean)

df_train['field16_mean_div'] = df_train['field16']/df_train['field16_mean']
df_train['field12_mean_div'] = df_train['field12']/df_train['field12_mean']
df_test['field16_mean_div'] = df_test['field16']/df_test['field16_mean']
df_test['field12_mean_div'] = df_test['field12']/df_test['field12_mean']

In [0]:
# mean encoding всех goal по field16
df_train['field16_cut'] = pd.cut(df_train['field16'], [-0.001,1,2,4,6,9,15,24,42,340])
df_test['field16_cut'] = pd.cut(df_test['field16'], [-0.001,1,2,4,6,9,15,24,42,340])
for targ in ['goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1']:
    map_dict = df_train.groupby('field16_cut')[targ].mean().to_dict()

    df_train['field16_mean_encoding_'+targ] = df_train['field16_cut'].map(map_dict).astype(np.float64)
    df_test['field16_mean_encoding_'+targ] = df_test['field16_cut'].map(map_dict).astype(np.float64)

In [0]:
df_train.drop('field16_cut', axis=1, inplace=True)
df_test.drop('field16_cut', axis=1, inplace=True)

In [0]:
# функция для составления признака - сочетание двух колонок
def all_mix_up(all_df, cols_to_mix):
    for i in tqdm_notebook(range(len(cols_to_mix))):
        for j in tqdm_notebook(range(i+1, len(cols_to_mix)), leave=False):
            c1 = cols_to_mix[i]
            c2 = cols_to_mix[j]
            new_col_name = c1 + '_mix_' + c2
            all_df[new_col_name] = all_df[c1].astype(str) + all_df[c2].astype(str)
            all_df[new_col_name] = all_df.groupby(new_col_name)['orderid'].transform('count')
    return all_df

In [0]:
def get_encode_features(train_df, test_df, cols_to_mix):
    all_df = pd.concat([train_df, test_df], ignore_index=True)

    all_df['field_0_value_encode'] = all_df.groupby('field0')['orderid'].transform('count')

    all_df['field_12_value_encode'] = all_df.groupby('field12')['orderid'].transform('count')
    all_df['field_16_value_encode'] = all_df.groupby('field16')['orderid'].transform('count')

    all_df['field_12_value_encode_userid'] = all_df.groupby(['userid', 'field12'])['orderid'].transform('count')
    all_df['field_16_value_encode_userid'] = all_df.groupby(['userid', 'field16'])['orderid'].transform('count')

    all_df['field16_mix_12'] = (all_df['field12'].astype(str) + all_df['field16'].astype(str)).astype(int)
    all_df['field7_mix_17_25'] = (all_df['field7'].astype(str) + all_df['field17'].astype(str) + all_df['field25'].astype(str)).astype(int)
    all_df['field15_mix_24'] = (all_df['field15'].astype(str) + all_df['field24'].astype(str)).astype(int)
    all_df['field2_mix_3'] = (all_df['field2'].astype(str) + all_df['field3'].astype(str)).astype(int)

    all_df = all_mix_up(all_df, cols_to_mix)

    return all_df[all_df.userid.isin(train_df.userid.unique())], all_df[all_df.userid.isin(test_df.userid.unique())]

In [0]:
df_train, df_test = get_encode_features(df_train, df_test, fields_cols)

In [0]:
df_train.replace({np.inf:-999, -np.inf:-999}, inplace=True)
df_test.replace({np.inf:-999, -np.inf:-999}, inplace=True)

In [0]:
df_train.fillna(-999, inplace=True)
df_test.fillna(-999, inplace=True)

In [0]:
for col in df_train.columns:
    if df_train[col].dtype == np.float64:
        df_train[col] = np.round(df_train[col], 6)

In [0]:
gc.collect()

0

### Вторая часть признаков, сделанная после НГ

In [0]:
def get_encode_features(train_df, test_df):
    
    all_df = pd.concat([train_df, test_df], ignore_index=True)

    # value_counts по userid
    fields_for_counts = ['field1', 'field6', 'field12', 'field13', 'field16', 'field22']
    for col in fields_for_counts:
        all_df[col+'_value_counts'] = all_df.groupby(col)['userid'].transform('count')
        gc.collect()

    # mean encoding по часу и дню недели отлета
    fields_for_mean = ['field1', 'field14', 'field25', 'field26', 'field27', 'field16']
    for col in fields_for_mean:
        all_df[col+'_mean_time_fly'] = all_df.groupby(['field3', 'field20', 'field23'])[col].transform('mean')
        all_df[col+'_std_time_fly'] = all_df.groupby(['field3', 'field20', 'field23'])[col].transform('std')
        all_df[col+'_mean_time_fly_div'] = all_df[col+'_mean_time_fly']/all_df[col]

    # mean encoding по количеству билетов и их цене
    fields_for_mean = ['field1_1', 'field22', 'field6', 'field13', 'field15', 'field17', 'field26',
                       'indicator_goal21', 'indicator_goal22', 'indicator_goal23', 'indicator_goal24', 'indicator_goal25']
    for col in fields_for_mean:
        all_df[col+'_mean_tickets'] = all_df.groupby(['field15', 'field14'])[col].transform('mean')
        all_df[col+'_mean_tickets_div'] = all_df[col+'_mean_tickets']/all_df[col]

    # процент билетов для взрослых
    all_df['tickets_grown_perc'] = all_df['field24']/all_df['field15']
    # цена одного билета
    all_df['price_of_tickects'] = all_df['field1_1']/all_df['field15']
    all_df['price_of_tickects_add'] = all_df['field1']/all_df['field15']
    # процент билетов для детей
    all_df['tickets_child_perc'] = (all_df['field28'])/all_df['field15']

    # функции добавляют mean encoding и min, max encoding переменных по другим переменным
    all_df = get_add_encode_mean(all_df)
    all_df = get_minmax_by_userid(all_df)
    all_df = get_encode_prices_mean(all_df)

    return all_df[all_df.userid.isin(train_df.userid.unique())], all_df[all_df.userid.isin(test_df.userid.unique())]

def get_add_encode_mean(all_df):
    features_to_encode = ['field1', 'field14', 'field25', 'field26', 'field27', 'field12', 'field16']
    features_by_encode = ['field1', 'field14', 'field25', 'field26', 'field27', 'field4', 'userid']
    for feat_by in tqdm_notebook(features_by_encode):
        for feat_to in features_to_encode:
            if feat_to != feat_by:
                all_df[feat_to+'_mean_encode_' + feat_by] = all_df.groupby(feat_by)[feat_to].transform('mean')
                all_df[feat_to+'_mean_encode_' + feat_by+'_div'] = all_df[feat_to]/all_df[feat_to+'_mean_encode_' + feat_by]
                gc.collect()
    return all_df

def get_minmax_by_userid(all_df):
    features_to_encode = ['field1', 'field14', 'field25', 'field26', 'field27', 'field12', 'field16']
    features_by_encode = [ 'userid']
    for feat_by in features_by_encode:
        for feat_to in tqdm_notebook(features_to_encode):
            if feat_to != feat_by:
                all_df[feat_to+'_max_encode_'+feat_by] = all_df.groupby(feat_by)[feat_to].transform('max')
                all_df[feat_to+'_max_encode_'+feat_by+'_div'] = all_df[feat_to]/all_df[feat_to+'_max_encode_'+feat_by]

                all_df[feat_to+'_min_encode_'+feat_by] = all_df.groupby(feat_by)[feat_to].transform('min')
                all_df[feat_to+'_min_encode_'+feat_by+'_div'] = all_df[feat_to]/all_df[feat_to+'_min_encode_'+feat_by]

                gc.collect()
    return all_df

def get_encode_prices_mean(all_df):
    features_by_encode = copy.copy(fields_cols) + copy.copy(ind_cols) + ['userid']
    features_by_encode.remove('field1_1')
    features_by_encode.remove('field14_1')
    features_to_encode = ['field1', 'field14', 'field27', 'field16', 'field12', 'hours_to_flight']
    for feat_by in tqdm_notebook(features_by_encode):
        for feat_to in features_to_encode:
            new_col = feat_to+'_mean_encode_' + feat_by
            if feat_to != feat_by and new_col not in all_df.columns :
                all_df[new_col] = all_df.groupby(feat_by)[feat_to].transform('mean')
                all_df[new_col+'_div'] = all_df[feat_to]/all_df[new_col]
                gc.collect()
    return all_df

In [0]:
df_train, df_test = get_encode_features(df_train, df_test)

### Предсказания модели

In [0]:
with open(os.path.join(data_folder, 'feature_for_task_2.txt'), 'r') as f:
    features = json.load(f)

In [0]:
variable_names = features['variable_names']

In [0]:
len(variable_names)

913

In [0]:
for col in tqdm_notebook(df_train.columns):
    df_train[col] = df_train[col].replace([np.inf, -np.inf], -9999)
    df_train[col] = df_train[col].fillna(-999)

    df_test[col] = df_test[col].replace([np.inf, -np.inf], -9999)
    df_test[col] = df_test[col].fillna(-999)

    gc.collect()

In [0]:
params = {'colsample_bytree': 0.8856,
 'learning_rate': 0.0113,
 'max_depth': 5,
 'n_estimators': 800,
 'num_leaves': 59,
 'reg_lambda': 0.1124,
 'subsample': 0.599}
 
lgb_cl = lgb.LGBMClassifier(**params, random_state=10, n_jobs=-1, importance_type='gain')

In [0]:
gc.collect()

0

In [0]:
targets = ['goal21', 'goal22', 'goal23', 'goal24', 'goal25']
for target in targets:
    lgb_cl.fit(df_train[variable_names], df_train[target])
    scr = lgb_cl.predict_proba(df_test[variable_names])[:,1]
    df_test[target] = np.round(scr,4)
    df_test[target] = df_test[target]*df_test['indicator_'+target]
    print(f'Target {target} learnt')

Target goal21 learnt
Target goal22 learnt
Target goal23 learnt
Target goal24 learnt
Target goal25 learnt


In [0]:
df_test[['orderid', 'goal21', 'goal22', 'goal23', 'goal24', 'goal25']].to_csv(os.path.join(subm_folder, 'task_2_submit.csv'), index=False)